In [ ]:
# 图定义计算过程，会话执行计算过程
# 为避免缓存变量冲突问题，每个模型都使用默认图， with tf.Graph().as_default():
# 图定义完成之后，可以保存下来，在会话内writer = tf.summary.FileWriter("logs/", sess.Graph)
# 保存模型：saver = tf.train.Saver()，在会话内，模型训练完成后：save_path = saver.save(sess, "my_net/demo.cpkt")
# 导入模型：
# 方式一：直接导入整个图模型，不需要重复定义之前的图
# saver = tf.train.import_meta_graph("my_net/demo.ckpt.meta"), 
# 在会话内：saver.restore(sess, "my_net/demo.ckpt")
# 方式二：需要重复定义之前的图
# saver = tf.train.Saver()
# 在会话内：saver.restore(sess, "my_net/demo.ckpt")

In [ ]:
# cross entropy loss
# loss = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(y_pred), reduction_indices=[1]))

# dropout
# def add_layer(xs, in_size, out_size, activation_function=None):
#     weights = tf.Variable(tf.random_normal([in_size, out_size], dtype=tf.float32))
#     biases = tf.Variable(tf.zeros([out_size])+0.1, dtype=tf.float32)
#     if activation_function is None:
#         ys = tf.nn.dropout(tf.matmul(xs, weights)+biases, keep_prob)
#     else:
#         ys = tf.nn.dropout(activation_function(tf.matmul(xs, weights)+biases))
#     return ys

# name scope
# def add_layer(xs, in_size, out_size, activation_function=None):
#     with tf.name_scope("layer"):
#         with tf.name_scope("weights"):
#             weights = tf.Variable(tf.random_normal([in_size, out_size], dtype=tf.float32))
#         with tf.name_scope("biases"):
#             biases = tf.Variable(tf.zeros([out_size])+0.1, dtype=tf.float32)
#         with tf.name_scope("ys"):
#             if activation_function is None:
#                 ys = tf.matmul(xs, weights)+biases
#             else:
#                 ys = activation_function(tf.matmul(xs, weights)+biases)
#             return ys     
# with tf.name_scope('inputs'):
#     xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
#     ys = tf.placeholder(tf.float32, [None, 1], name='y_input')
# with tf.name_scope('loss'):
#     loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
#                                         reduction_indices=[1]))
# with tf.name_scope('train'):
#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# tensorboard
# with tf.Session() as sess:
#     writer = tf.summary.FileWriter("logs/", sess.graph)
#     sess.run(init) 
# tensorboard --logdir="./logs"

# save the network
# init= tf.global_variables_initializer()
# saver = tf.train.Saver()
# with tf.Session() as sess:
#    sess.run(init)
#    save_path = saver.save(sess, "my_net/save_net.ckpt")

# restore the network
# saver = tf.train.Saver()
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     saver.restore(sess, "my_net/save_net.cktp")

# save the network
# W = tf.Variable([[1,2,3],[3,4,5]], dtype=tf.float32, name='weights')
# b = tf.Variable([[1,2,3]], dtype=tf.float32, name='biases')
# init= tf.initialize_all_variables()
# saver = tf.train.Saver()
# with tf.Session() as sess:
#    sess.run(init)
#    save_path = saver.save(sess, "my_net/save_net.ckpt")
#    print("Save to path: ", save_path)

# restore the network
# W = tf.Variable(np.arange(6).reshape((2, 3)), dtype=tf.float32, name="weights")
# b = tf.Variable(np.arange(3).reshape((1, 3)), dtype=tf.float32, name="biases")
# # not need init step
# saver = tf.train.Saver()
# with tf.Session() as sess:
#     saver.restore(sess, "my_net/save_net.ckpt")
#     print("weights:", sess.run(W))
#     print("biases:", sess.run(b))

In [1]:
import numpy as np
import tensorflow as tf

In [11]:
with tf.Graph().as_default():

    x = np.float32(np.linspace(-1.0, 1.0, 300)[:, np.newaxis])
    noise = np.random.normal(0, 0.05, x.shape)
    y = np.square(x)-0.5+noise
    
    with tf.name_scope("placeholder"):
        xs = tf.placeholder(shape=[300, 1], dtype=tf.float32, name="xs")
        ys = tf.placeholder(shape=[300, 1], dtype=tf.float32, name="ys")
    with tf.name_scope("l1"):
        w1 = tf.Variable(tf.random_normal(shape=[1, 10], dtype=tf.float32), name="w1")
        b1 = tf.Variable(tf.zeros(shape=[10])+0.1, dtype=tf.float32, name="b1")
        l1 = tf.nn.relu(tf.matmul(xs, w1)+b1)
    with tf.name_scope("l2"):
        w2 = tf.Variable(tf.random_normal(shape=[10, 1]), dtype=tf.float32, name="w2")
        b2 = tf.Variable(tf.zeros(shape=[1])+0.1, dtype=tf.float32, name="b2")
        y_pred = tf.matmul(l1, w2)+b2
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-y_pred), reduction_indices=[1]))
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    train = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)
        writer = tf.summary.FileWriter("logs/", sess.graph)
        for step in range(1001):
            sess.run(train, feed_dict={xs:x, ys:y})
            if step%50 == 0:
                print("step: ", step, sess.run(loss, feed_dict={xs:x, ys:y}))
        saver.save(sess, "my_net/demo.ckpt")

step:  0 0.575654
step:  50 0.0066488246
step:  100 0.004886212
step:  150 0.0042625237
step:  200 0.0039244806
step:  250 0.0036874234
step:  300 0.0035542662
step:  350 0.0034145887
step:  400 0.003331141
step:  450 0.0032769945
step:  500 0.0032315303
step:  550 0.003187273
step:  600 0.0031461902
step:  650 0.0031141457
step:  700 0.0030870815
step:  750 0.0030574347
step:  800 0.0030265069
step:  850 0.0029934675
step:  900 0.00296094
step:  950 0.0029298943
step:  1000 0.0028999264


In [22]:
# 只载入参数，无需重复定义图
with tf.Graph().as_default():
    saver = tf.train.import_meta_graph("my_net/demo.ckpt.meta")
    with tf.Session() as sess:
        saver.restore(sess, "my_net/demo.ckpt")
        print("w1: ", sess.run(tf.get_default_graph().get_tensor_by_name("l1/w1:0")))
        print("b1: ", sess.run(tf.get_default_graph().get_tensor_by_name("l1/b1:0")))
        print("w2: ", sess.run(tf.get_default_graph().get_tensor_by_name("l2/w2:0")))
        print("b2: ", sess.run(tf.get_default_graph().get_tensor_by_name("l2/b2:0")))

INFO:tensorflow:Restoring parameters from my_net/demo.ckpt
w1:  [[ 0.53038085 -1.0457807   0.12552173  0.91430354  0.6683802  -1.4113299
   0.6124318  -0.29901764  0.02164143  0.594167  ]]
b1:  [ 0.08576133 -1.478195    0.06166786 -0.28530255  0.10620202 -0.6543978
  0.19150573 -0.32496488  0.1790835   0.09040033]
w2:  [[ 2.3643507e-01]
 [ 2.0161088e+00]
 [-1.5492942e+00]
 [ 1.4774841e+00]
 [-1.8678739e-03]
 [ 1.0833520e+00]
 [-2.9598641e-01]
 [ 4.8799190e-01]
 [-4.9223500e-01]
 [ 3.5328797e-01]]
b2:  [-0.2777171]


In [21]:
# 载入整个图，需要重复定义图
with tf.Graph().as_default():
    
    x = np.float32(np.linspace(-1.0, 1.0, 300)[:, np.newaxis])
    noise = np.random.normal(0, 0.05, x.shape)
    y = np.square(x)-0.5+noise
    
    with tf.name_scope("placeholder"):
        xs = tf.placeholder(shape=[300, 1], dtype=tf.float32, name="xs")
        ys = tf.placeholder(shape=[300, 1], dtype=tf.float32, name="ys")
    with tf.name_scope("l1"):
        w1 = tf.Variable(tf.random_normal(shape=[1, 10], dtype=tf.float32), name="w1")
        b1 = tf.Variable(tf.zeros(shape=[10])+0.1, dtype=tf.float32, name="b1")
        l1 = tf.nn.relu(tf.matmul(xs, w1)+b1)
    with tf.name_scope("l2"):
        w2 = tf.Variable(tf.random_normal(shape=[10, 1]), dtype=tf.float32, name="w2")
        b2 = tf.Variable(tf.zeros(shape=[1])+0.1, dtype=tf.float32, name="b2")
        y_pred = tf.matmul(l1, w2)+b2
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-y_pred), reduction_indices=[1]))
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    train = optimizer.minimize(loss)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, "my_net/demo.ckpt")
        print("w1: ", sess.run(tf.get_default_graph().get_tensor_by_name("l1/w1:0")))
        print("b1: ", sess.run(tf.get_default_graph().get_tensor_by_name("l1/b1:0")))
        print("w2: ", sess.run(tf.get_default_graph().get_tensor_by_name("l2/w2:0")))
        print("b2: ", sess.run(tf.get_default_graph().get_tensor_by_name("l2/b2:0")))
        print("loss: ", sess.run(loss, feed_dict={xs:x, ys:y}))

INFO:tensorflow:Restoring parameters from my_net/demo.ckpt
w1:  [[ 0.53038085 -1.0457807   0.12552173  0.91430354  0.6683802  -1.4113299
   0.6124318  -0.29901764  0.02164143  0.594167  ]]
b1:  [ 0.08576133 -1.478195    0.06166786 -0.28530255  0.10620202 -0.6543978
  0.19150573 -0.32496488  0.1790835   0.09040033]
w2:  [[ 2.3643507e-01]
 [ 2.0161088e+00]
 [-1.5492942e+00]
 [ 1.4774841e+00]
 [-1.8678739e-03]
 [ 1.0833520e+00]
 [-2.9598641e-01]
 [ 4.8799190e-01]
 [-4.9223500e-01]
 [ 3.5328797e-01]]
b2:  [-0.2777171]
loss:  0.0034549555
